In [51]:
import json
import pandas as pd
import re

In [52]:
with open('tioberny.txt', 'r', encoding='utf-16') as file:
    tweets = [json.loads(line) for line in file]

tioberny_tweets = pd.DataFrame(tweets)

In [53]:
with open('traficogt.txt', 'r', encoding='utf-16') as file:
    tweets = [json.loads(line) for line in file]

traficogt_tweets = pd.DataFrame(tweets)


In [54]:
print(tioberny_tweets.columns)
print(traficogt_tweets.columns)


Index(['id', 'id_str', 'url', 'date', 'user', 'lang', 'rawContent',
       'replyCount', 'retweetCount', 'likeCount', 'quoteCount',
       'bookmarkedCount', 'conversationId', 'conversationIdStr', 'hashtags',
       'cashtags', 'mentionedUsers', 'links', 'viewCount', 'retweetedTweet',
       'quotedTweet', 'place', 'coordinates', 'inReplyToTweetId',
       'inReplyToTweetIdStr', 'inReplyToUser', 'source', 'sourceUrl',
       'sourceLabel', 'media', 'card', '_type'],
      dtype='object')
Index(['id', 'id_str', 'url', 'date', 'user', 'lang', 'rawContent',
       'replyCount', 'retweetCount', 'likeCount', 'quoteCount',
       'bookmarkedCount', 'conversationId', 'conversationIdStr', 'hashtags',
       'cashtags', 'mentionedUsers', 'links', 'viewCount', 'retweetedTweet',
       'quotedTweet', 'place', 'coordinates', 'inReplyToTweetId',
       'inReplyToTweetIdStr', 'inReplyToUser', 'source', 'sourceUrl',
       'sourceLabel', 'media', 'card', '_type'],
      dtype='object')


Convertir a minúsculas

In [55]:
tioberny_tweets['rawContent'] = tioberny_tweets['rawContent'].str.lower()
traficogt_tweets['rawContent'] = traficogt_tweets['rawContent'].str.lower()


Quitar URLs

In [56]:
def remove_urls(text):
    return re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

tioberny_tweets['rawContent'] = tioberny_tweets['rawContent'].apply(remove_urls)
traficogt_tweets['rawContent'] = traficogt_tweets['rawContent'].apply(remove_urls)


Quitar emoticones

In [57]:
def remove_emoticons(text):
    return re.sub(r'[^\w\s]', '', text)

tioberny_tweets['rawContent'] = tioberny_tweets['rawContent'].apply(remove_emoticons)
traficogt_tweets['rawContent'] = traficogt_tweets['rawContent'].apply(remove_emoticons)


Quitar signos de puntuación (exceptuando "@" y "#")

In [60]:
tioberny_tweets['rawContent'] = tioberny_tweets['rawContent'].str.replace(r'[^\w\s@#]', '', regex=True)
traficogt_tweets['rawContent'] = traficogt_tweets['rawContent'].str.replace(r'[^\w\s@#]', '', regex=True)


In [61]:
# Crear listas para almacenar las interacciones
interactions = []

for index, row in tioberny_tweets.iterrows():
    user_id = row['user']['id_str']
    text = row['rawContent']
    
    # Extraer menciones
    mentions = re.findall(r'@(\w+)', text)  # Extraer nombres de usuario mencionados
    mentions = [mention.lower() for mention in mentions]  # Normalizar
    
    # Comprobar si es una respuesta o un retweet
    is_reply = row['inReplyToUser'] is not None
    is_retweet = row['retweetedTweet'] is not None

    interactions.append({
        'user_id': user_id,
        'mentions': mentions,
        'is_reply': is_reply,
        'is_retweet': is_retweet,
        'text': text
    })

interactions_df = pd.DataFrame(interactions)


In [62]:
# Eliminar duplicados
interactions_df.drop_duplicates(subset=['user_id', 'text'], inplace=True)

# Normalizar menciones
interactions_df['mentions'] = interactions_df['mentions'].apply(lambda x: [mention.lower() for mention in x])


In [64]:
!pip install networkx

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.7 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 7.1 MB/s eta 0:00:00


In [63]:
import networkx as nx

# Crear un grafo dirigido
G = nx.DiGraph()

# Agregar nodos y aristas
for _, row in interactions_df.iterrows():
    user_id = row['user_id']
    mentions = row['mentions']
    
    # Añadir nodo para el usuario
    if not G.has_node(user_id):
        G.add_node(user_id)

    # Añadir aristas por cada mención
    for mention in mentions:
        G.add_edge(user_id, mention)  # Interacción dirigida

# Convertir a una matriz de adyacencia si es necesario
adjacency_matrix = nx.to_numpy_array(G)


ModuleNotFoundError: No module named 'networkx'